# Adversarial

- Adversarial input to emphasize benefit of our algorithm, even online

Schedulers:
```
schedulers = [
        MRIS(sort='WSVF'),
        OnlinePriorityQueue(sort='WSVF'),
        TetrisScheduler(),
        BFEXECScheduler()
]
```

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scienceplots
import itertools
import matplotlib
from pathlib import Path

In [2]:
plt.style.use(['science'])
params = {
    "font.family": "serif",
    "text.usetex": True,
    'text.latex.preamble':
        r"""
        \usepackage{libertine}
        \usepackage[libertine]{newtxmath}
        """,
}
# https://matplotlib.org/stable/gallery/lines_bars_and_markers/linestyles.html
def get_linestyles():
     return itertools.cycle([
         ('solid', (0, ())),
         # ('loosely dotted',        (0, (1, 10))),
         ('dotted',                (0, (1, 1))),
         # ('densely dotted',        (0, (1, 1))),
         ('long dash with offset', (5, (10, 3))),
         ('loosely dashed',        (0, (5, 10))),
         #('dashed',                (0, (5, 5))),
         ('densely dashed',        (0, (5, 1))),
    
         #('loosely dashdotted',    (0, (3, 10, 1, 10))),
         ('dashdotted',            (0, (3, 5, 1, 5))),
         ('densely dashdotted',    (0, (3, 1, 1, 1))),
    
         ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
         ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
         ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1)))
    ])
mpl.rcParams.update(params)

# Input Parameters

In [3]:
# Number of resources
R = 3

# Number of jobs
N = 2500

# Number of machines
M = 1

# The particular run number analyze
RUN_NUMBER = 1

# Path to the results folder
PATH_TO_RESULTS = Path('./results/')

In [4]:
df = pd.read_parquet(PATH_TO_RESULTS / Path(f'6_adversarial_{RUN_NUMBER}.parquet'))
schedulers = df['scheduler'].unique().tolist()

In [5]:
schedulers

['MRIS-WSVF', 'OnlinePQ-WSVF', 'Tetris-instantaneous', 'BF-EXEC']

# Plot the profile of each scheduler

In [31]:
resource_idx = 0
demand_levels = 10
fig, axs = plt.subplots(nrows=2, ncols=2, tight_layout=True, dpi=600)
fig.set_figheight(3)
fig.set_figwidth(6)

# gradient = np.linspace(0, 1, N+1)
# idx = np.arange(0, N+1)
# np.random.shuffle(idx)
# colormap = plt.cm.gist_rainbow
# colors = colormap(gradient)

# axs return value depends on the input rows and cols. Convert to always use 2D ndarray
if type(axs) is not np.ndarray:
    axs = np.array([[axs]])
elif axs.ndim == 1:
    axs = axs.reshape(1, -1)

axes = axs.flat
    
for s, scheduler in enumerate(schedulers):
    if scheduler == 'Tetris-instantaneous':
        label = r'\textsc{Tetris}'
    elif scheduler == 'PQ-WSVF':
        label = 'CA-PQ-WSVF'
    elif scheduler == 'OnlinePQ-WSVF':
        label = 'PQ-WSVF'
    else:
        label = scheduler
        
    df_ = df[df['scheduler'] == scheduler]
    start_times = df_['S'].to_numpy()
    completion_times = df_['C'].to_numpy()
    demands = df_['d'].to_numpy()
    
    x = np.concatenate([np.zeros(1), start_times, completion_times, start_times+1E-12, completion_times-1E-12])
    x = np.unique(x)
    
    x = np.sort(x)
    cumulative_resource = np.zeros_like(x)
    
    stacked_data = []
    
    for i in range(N):
        cumulative_resource += np.where((start_times[i] < x) & (completion_times[i] > x), demands[i][resource_idx], 0) / demand_levels
        # cumulative_resource += np.where(((start_times[i]) <= x) & (completion_times[i] > x), demands[i][resource_idx], 0) / demand_levels
        stacked_data.append(np.copy(cumulative_resource))
    stacked_data.reverse()
    for j, data in enumerate(stacked_data):
        axes[s].stackplot(x, data, rasterized=True)
        axes[s].set_ylim(0, 1.1)
        axes[s].set_title(label)
        
fig.text(0.5, -0.00, 'Time [arb. units]', ha='center')
fig.text(-0.01, 0.5, 'Resource usage [arb. units]', va='center', rotation='vertical')
# plt.savefig('adversarial.png', bbox_inches='tight', format='png')
# plt.savefig('adversarial.pdf', bbox_inches='tight')
# plt.savefig('adversarial.svg', bbox_inches='tight', format='svg')
plt.savefig('adversarial.pdf', bbox_inches='tight', format='pdf')
plt.close()
# plt.show()

# AWCT Metrics

In [ ]:
awct_data = {}
for scheduler in schedulers:
    df_ = df[df['scheduler'] == scheduler]
    awct = df_['C'].dot(df_['w']) / len(df)
    awct_data[scheduler] = awct

plt.figure(figsize=(10, 5), dpi=200)
bars = plt.barh([str(scheduler) for scheduler in schedulers], awct_data.values(), align='center')
# autolabel_h(bars, scientific_notation=True)
plt.margins(x=0.10)
plt.title(f'One-Shot Comparison - $N={N}$, $M={M}$, $R={R}$')
plt.xlabel(r"Average Weighted Completion Time")
plt.ylabel("Scheduler")
plt.tight_layout()
# plt.savefig(f'images/experiment9_{dataset.dataset}_{num_jobs}.png')
plt.show()